Project 1 requires you to create a set of FSAs, one for each legal token type allowed in the _Datalog_ database language. Here are the first several token types from the project.

| Token Type    | Description           |  
| :--           | :--                   |  
| COMMA         | The ',' character     |
| PERIOD        | The '.' character     |
| Q_MARK        | The '?' character     |
| LEFT_PAREN    | The '(' character     |
| RIGHT_PAREN   | The ')' character     |
| COLON         | The ':' character     |
| COLON-DASH    | The string ":-"       |

This means that you will have a collection of FSAs that you will need to manage. 

***

One way to implement project 1 is to run each FSA on the input string, as illustrated in the left side of the figure. 

![image](Managing_project1_FSAs.drawio.svg)

Each FSA outputs accept or reject, represented by the T or F, respectively on the rightside of the collection of FSAs.

Each output can be collected into some sort of list, and that list is the input to a finite state machine (FSM) that decides which token is output. Notice how the arrows on the FSM on the right have two components: an input, which is the list represented by the sequence of true/false values in the square brackets, and an output, which is the token. The input and output are separated by a comma, which is how the textbook differentiates between the input and output in the FSMs it draws.

The way this FSM operates is that it reads the list of true/false values and outputs the correct token. The top loop in the FSM says that the FSA for the left parenthesis succeeded and all others failed [TFFF], so the output should be the LEFT_PAREN token. The bottom right loop says that both the colon and colon-dash FSAs succeeded [FFTT], so the output should be the COLON-DASH token.

***
We can now start to think about how to program the collection of FSAs and the manager FSM.  First, define the base_class from which all FSAs inherit. The code below is copied straigh from the previous Jupyter notebook tutorial.

In [7]:
class FSA:
    """ FSA Base or Super class"""
    def __init__(self,name):
        """ Class constructor """
        ###############################################################
        # Define the five elements of the FSA
        ###############################################################
        # Set of states: Each state S0, S1, S2, Serr will be represented by its own method
        # Set of inputs: I is the set of alphanumeric characters, checked by isalnum()
        self.start_state = self.S0  # We'll always have the start state named S0
        self.accept_states = set()  # No accept states defined
        # Transition function: Each transition will be defined in the state methods
        
        ###############################################################
        # Define four variables that are used within the FSA, some
        # to make the FSA run and some that help us understand how 
        # the FSA works
        ###############################################################
        self.input_string = ""      # Default empty input
        self.FSA_name = name
        self.num_chars_read = 0
            
    #########################################################
    # Define each state as a function                       #
    # Within each function, define the transition function. #
    # Each transition function reads the current input and  #
    # choose the next state based on the input              #
    #########################################################
    def S0(self):
        """ Every FSA must have a start state, and we'll always name 
        it S0. The method for the start state must be defined in the
        derived class since it's not defined here. """
        raise NotImplementedError()     # This line causes an error to 
                                        # occur if the child classes don't implement this method

    ############################
    # Public Manager Functions
    ############################
    def Run(self,input_string):
        ###############################################################
        # This function will be called to make the FSA execute
        # It records the input string,
        # sets the current state to the start state
        # and then calls a state method that returns the next state
        # Each state method accesses the input_string
        ###############################################################
            # Remember input_string
        self.input_string = input_string
            # Set current state to start state
        current_state = self.start_state
            # Call current state, which starts the FSA
        while self.num_chars_read < len(self.input_string):
            current_state = current_state()
            # Check whether the FSA ended in an accept state
        outcome = False 
        if current_state in self.accept_states: outcome = True # Accept if the FSA ended in an accept state
        return outcome
    def Reset(self):
        self.num_chars_read = 0
        self.history = []
    
    ############################
    # Public Getters and Setters
    ############################
    def get_Name(self): return self.FSA_name
    def set_Name(self,FSA_name): self.FSA_name = FSA_name
    
    ############################
    # Private Helper functions
    ############################
    def __getCurrentInput(self):  # The double underscore makes the method private
        current_input = self.input_string[self.num_chars_read]
        self.num_chars_read += 1
        return current_input

***
We now define an FSA class for each of the FSAs in the figure above. The Colon and ColonDash FSAs are copied from the previous tutorial.

In [8]:
class ColonDash_FSA(FSA):
    def __init__(self):
        FSA.__init__(self,"colonDash_FSA") # You must invoke the __init__ of the parent class
        self.accept_states.add(self.S2) # Since self.accept_states is defined in parent class, I can use it here
    
    def S0(self):
        current_input = self._FSA__getCurrentInput()
        if current_input == ':': next_state = self.S1
        else: next_state = self.Serr
        return next_state
    def S1(self):
        current_input = self._FSA__getCurrentInput()
        if current_input == '-': next_state = self.S2
        else: next_state = self.Serr
        return next_state
    def S2(self):
        current_input = self._FSA__getCurrentInput()
        next_state = self.S2 # loop in state S2
        return next_state
    def Serr(self):
        current_input = self._FSA__getCurrentInput()
        next_state = self.Serr # loop in state Serr
        return next_state

***

In [9]:
class Colon_FSA(FSA):
    def __init__(self):
        FSA.__init__(self,"colon_FSA") # You must invoke the __init__ of the parent class
        self.accept_states.add(self.S1) # Since self.accept_states is defined in parent class, I can use it here
    
    def S0(self):
        current_input = self._FSA__getCurrentInput()
        if current_input == ':': next_state = self.S1
        else: next_state = self.Serr
        return next_state
    def S1(self):
        current_input = self._FSA__getCurrentInput()
        next_state = self.S1 # loop in state S1
        return next_state
    def Serr(self):
        current_input = self._FSA__getCurrentInput()
        next_state = self.Serr # loop in state Serr
        return next_state

***
The FSA for left and right parentheses tokens follow the pattern for the colon token exactly

In [10]:
class LeftParen_FSA(FSA):
    def __init__(self):
        FSA.__init__(self,"leftPAren_FSA") # You must invoke the __init__ of the parent class
        self.accept_states.add(self.S1) # Since self.accept_states is defined in parent class, I can use it here
    
    def S0(self):
        current_input = self._FSA__getCurrentInput()
        if current_input == '(': next_state = self.S1
        else: next_state = self.Serr
        return next_state
    def S1(self):
        current_input = self._FSA__getCurrentInput()
        next_state = self.S1 # loop in state S1
        return next_state
    def Serr(self):
        current_input = self._FSA__getCurrentInput()
        next_state = self.Serr # loop in state Serr
        return next_state


***

In [11]:
class RightParen_FSA(FSA):
    def __init__(self):
        FSA.__init__(self,"rightPAren_FSA") # You must invoke the __init__ of the parent class
        self.accept_states.add(self.S1) # Since self.accept_states is defined in parent class, I can use it here
    def S0(self):
        current_input = self._FSA__getCurrentInput()
        if current_input == ')': next_state = self.S1
        else: next_state = self.Serr
        return next_state
    def S1(self):
        current_input = self._FSA__getCurrentInput()
        next_state = self.S1 # loop in state S1
        return next_state
    def Serr(self):
        current_input = self._FSA__getCurrentInput()
        next_state = self.Serr # loop in state Serr
        return next_state

*** 
We've implemented each FSA on the lefthand side of the figure above in their own classes using inheritance. Now we need to figure out how to run each FSA on the same input and collect the outputs. Let's use a _dictionary_ data structure to represent the collection of FSAs. Begin by creating instances of each FSA type.


In [12]:
right_paren_FSA = RightParen_FSA()   # I'm using my own made up naming convention. It's not very good. Look for better styles.
left_paren_FSA = LeftParen_FSA()     # The convention is that classes have capital letters and instances use all lower case. uggh
colon_FSA = Colon_FSA()
colon_dash_FSA = ColonDash_FSA()

The dictionary will be indexed (keyed) by the instances of the FSAs, and the dictionary values will be the return status of each FSA when run on the input. Chat GPT gives a good response to the prompt _how do i create a dictionary with only keys in python_, but something kinda bugs me about Chat GPT: it's using data generated by people who have posted information on the internet but it doesn't attribute credit to them. So, I put the same prompt into google and am using one of the responses from the geeksforgeeks tutorial https://www.geeksforgeeks.org/python-initialize-a-dictionary-with-only-keys-from-a-list/ .

In [13]:
FSA_keys = [right_paren_FSA,left_paren_FSA,colon_FSA,colon_dash_FSA]
FSADict = dict.fromkeys(FSA_keys, False)  # Initialize the outputs from each FSA to false
print(FSADict)

{<__main__.RightParen_FSA object at 0x10d59f950>: False, <__main__.LeftParen_FSA object at 0x10d59ef90>: False, <__main__.Colon_FSA object at 0x10d59f650>: False, <__main__.ColonDash_FSA object at 0x10d548490>: False}


I'm taking advantage of something that I like in Python, specifically that methods and instantiated classes are treated as _first-class objects_. Quoting from Chat GPT in response to the prompt _why can we call methods first order elements in python_:

    In Python, methods are first-class objects, which means that they can be treated as any other object, such as variables, data types, and functions. This means that we can pass methods as arguments to other functions, return methods from functions, and store methods in data structures like lists and dictionaries.

That means that we can use the name of the FSA objects as keys to the dictionary. 